In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = 'data/train_label.csv'

In [ ]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [ ]:
num_class = len(train_label['label'].unique())

In [ ]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [ ]:
# train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, 
#                            event_time=10, nrows=120)


In [ ]:
train = joblib.load('data/df_train_10_60.pkl').reset_index()
test = joblib.load('data/df_test_10.pkl')


In [ ]:
y_cols = 'label'
fea_cols = [c for c in train.columns if c[0] == 'V']
len(fea_cols), y_cols

In [ ]:
scaler = joblib.load('scaler_20200129T135731.bin')

train[fea_cols] = scaler.transform(train[fea_cols].values)
test[fea_cols] = scaler.transform(test[fea_cols].values)

In [ ]:
zero_cols = joblib.load('zero_cols.bin')
fea_cols = [c for c in fea_cols if c not in zero_cols]
input_size = len(fea_cols)
input_size

In [ ]:
# for c in fea_cols:
#     if len(train[c].unique()) == 1:
#         print(c)

In [ ]:
train

In [ ]:
df_label = pd.read_csv('data/train_label.csv', index_col=0)
df_label

id2label = df_label.to_dict()['label']
id2label

label2id = {}
for k, v in id2label.items():
    if v in label2id.keys():
        label2id[v].add(k)
    else:
        label2id[v] = set()
        label2id[v].add(k)

id2label

In [ ]:
fea_cols

In [ ]:
x_dict = {}

for i in train['index'].unique():
    x_dict[i] = [np.stack(train[train['index'] == i][fea_cols].values)]

df_seq = pd.DataFrame.from_dict(x_dict, orient='index', columns=['seq'])

df_seq['label'] = df_seq.index.map(lambda x: id2label[x])

df_seq

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=81511991154 % 2**32-1)

# X_train.shape, X_test.shape

In [ ]:
class Dataset15(Dataset):
    def __init__(self, df, seq_col, y_col):        
#         self.X = df[fea_cols].values
        self.X = np.stack(df_seq['seq'].values)
#         self.y = pd.get_dummies(df[y_cols]).values
        self.y = df[y_col].values

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
#         print(idx)
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.long)
    

In [ ]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def set_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def set_scheduler(self, scheduler):
        self.scheduler = scheduler
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred.size(), y_batch.size())
            
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        self.scheduler.step()
        
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                total_loss = total_loss + loss.item()
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
#         torch.save(self.model.state_dict(), 'checkpoint.pt')
        joblib.dump(self.model, model_path)
        return
    
    def load(self, model_path='checkpoint.pt'):
#         self.model.load_state_dict(torch.load(model_path))
        self.model = joblib.load(model_path)
        return

#### Model

In [ ]:
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, device, num_directions=1):
        super(RNNModel, self).__init__()

        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers        
        self.num_directions = num_directions

        
        self.lstm = torch.nn.LSTM(input_size,
                                  hidden_size,
                                  num_layers,
                                  batch_first=True,
                                  bidirectional=(num_directions==2))
        
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=0.0001)
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * self.num_directions, 198)
            
#             torch.nn.Linear(hidden_size, 64), 
#             relu, 
#             torch.nn.BatchNorm1d(64), 
#             dropout,
#             torch.nn.Linear(64, 2)
        )
        
        

    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions, batch_size, self.hidden_size)).to(self.device)
        cell = Variable(torch.zeros(self.num_layers*self.num_directions, batch_size, self.hidden_size)).to(self.device)
        return hidden, cell

    def forward(self, x_seq):
#         embed = self.embed(x_seq) # word vector indexing
        hidden, cell = self.init_hidden(x_seq.size(0)) # initial hidden,cell
        
        output, (hidden, cell) = self.lstm(x_seq, (hidden, cell))
        
        # Many-to-Many
#         output = self.fc(output) # B,T,H -> B,T,V
        
        # Many-to-One
        hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        hidden = torch.cat([h for h in hidden], 1)
        output = self.fc(hidden) # last hidden
        
        return nn.LogSoftmax(dim=1)(output)
    

#### Train

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

# print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')

In [ ]:
torch.manual_seed(81511991154)
torch.initial_seed()

In [ ]:
seq_col = 'seq'
y_col = 'label'

In [ ]:
dataset = Dataset15(df_seq, seq_col, y_col)

train_size = int(df_seq.shape[0] * 0.8)
val_size = df_seq.shape[0] - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

print(len(train_set), len(val_set))

batch_size = 25
num_workers = 8

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=1)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

In [ ]:
len(dataset)

In [ ]:
# model = DNNModel(input_size=len(fea_cols), dropout_probability=0.7).to(device)
# model = CNNModel(dropout_probability=0.7).to(device)
model = RNNModel(input_size=input_size, hidden_size=1024, 
                 num_layers=1, device=device, num_directions=1).to(device)


In [ ]:
val_loss_min = np.Inf

In [ ]:
# criterion(torch.Tensor([[0,0,1]]), torch.Tensor([[0.1,0.2, 1]]))

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean').to(device)
# criterion = nn.MultiLabelSoftMarginLoss().to(device)

criterion = nn.NLLLoss().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=50, gamma=1.0)

trainer = Trainer(model, criterion, optimizer, scheduler, device)



##### LR

In [ ]:
lr_list = [
#     (0.1, 20),
    (0.01, 50),
    (0.003, 50),
    (0.001, 50),
    (0.0003, 50),
    (0.0001, 50),
    (0.00003, 50),
    (0.00001, 50),
    (0.000005, 50),
]


##### Trainer

In [ ]:
total_epoch = 10000

for lr, patience in lr_list:
    print(lr, patience)
    if os.path.isfile('stop.flag'):
        print('stop!')
        break
    
    early_stopping = EarlyStopping(patience=patience, min_epoch=1, verbose=True)
    early_stopping.val_loss_min = val_loss_min
    early_stopping.best_score = None if val_loss_min==np.Inf else -val_loss_min 
    
    trainer.optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     trainer.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
#     trainer.optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    trainer.scheduler = StepLR(trainer.optimizer, step_size=50, gamma=1.0)
    
    for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
        if os.path.isfile('stop.flag'):
            print(f'{e} stop!')
            break

        train_loss = trainer.train(train_loader)
        
        if e % 1 == 0:
            valid_loss = trainer.eval(val_loader)
    #         valid_loss = train_loss

            ts = datetime.now().strftime('%Y%m%dT%H%M%S')
            print(f'[{ts}] Epock {e} / {total_epoch}\t lr {trainer.scheduler.get_lr()[0]}')
            print(f'  train_loss: {train_loss}  valid_loss: {valid_loss}')

            early_stopping(valid_loss, model)

            if early_stopping.early_stop:
                print("\tEarly stopping epoch {}, valid loss {}".format(e, early_stopping.val_loss_min))
                break
            

    model.load_state_dict(torch.load('model/checkpoint.pt'))
#     trainer.load('model/checkpoint.pt')
    val_loss_min = early_stopping.val_loss_min
    
    
    model_path = 'model/{}_{}'.format(model_ts, val_loss_min)
#     joblib.dump(model, '{}.model'.format(model_path))
#     torch.save(model.state_dict(), '{}.pt'.format(model_path))
    trainer.save('{}.model'.format(model_path))
    print(model_path)

    # torch.save(model.state_dict(), f'checkpoint.pt.{train_loss}')
    

In [ ]:
# model = joblib.load('model/20200202T232911_0.7551849484443665.model').cpu()

In [ ]:
# df_test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=None)
print(test.shape)
# test[fea_cols] = scaler.transform(test[fea_cols].values)

model.eval()
y_pred = model(torch.Tensor(test[fea_cols].values))    


In [ ]:
y_pred = F.softmax(y_pred)

print(y_pred)

In [ ]:
# model_tag ='{}_{}_{}'.format(model_ts, train_loss, valid_loss)
model_tag ='{}'.format(model_ts)

In [ ]:
submission = pd.DataFrame(data=y_pred.cpu().detach().numpy())
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()

submission.to_csv('submit/{}.csv'.format(model_tag), index=True) 
model_tag

In [ ]:
submission